In [32]:
import cv2
import random
import numpy as np
import math
import statistics
import pickle
from PIL import Image, ImageOps
from tqdm import tqdm
import matplotlib.pyplot as plt
import scipy.linalg
from collections import Counter
from scipy.spatial import distance
import warnings
warnings.simplefilter("ignore", RuntimeWarning)

In [33]:
img_read_1 = Image.open(r"Q3_2_SIFT.jpg") 
img_read_1 = img_read_1.resize((256, 256))
img_read_1.save("Data_SIFT_rgb.png")
img_rgb_array = np.array(img_read_1)
img_gray_1 = ImageOps.grayscale(img_read_1)
img_array_1 = np.array(img_gray_1)
temp_pad_1 = np.pad(img_array_1, pad_width=1, mode='constant', constant_values=0)

In [34]:
count_scales = 5
window_size = 3
value_sigma = 1.5
list_np_array = []
value_scaling_factor = pow(2,0.25)

#Applying Gaussian Blur on the five scales
for a in range(count_scales):
    img_output = cv2.GaussianBlur(img_array_1,(window_size,window_size),value_sigma)
    list_np_array.append(img_output)
    value_sigma = value_sigma * value_scaling_factor

In [35]:
list_DOG = []
for b in range(len(list_np_array) - 1):
    arr_scale_1 = list_np_array[b]
    arr_scale_2 = list_np_array[b+1]
    arr_DOG = np.subtract(arr_scale_1, arr_scale_2)
    list_DOG.append(arr_DOG)

In [36]:
#Now finding the pixels which are a maxima or a minima
list_max_min_cord = []
for c in range(1,len(list_DOG)-1):
    #Now picking each pixel in that particulaer DoG
    curr_array = list_DOG[c]
    for row_num in range(1, curr_array.shape[0] - 1):
        for col_num in range(1, curr_array.shape[1] - 1):
            curr_pixel_val = curr_array[row_num][col_num]
            list_neigh_values = []
            list_neigh_values.append(curr_array[row_num][col_num-1])
            list_neigh_values.append(curr_array[row_num][col_num+1])
            row_abv = curr_array[row_num-1,col_num-1:col_num+2]
            row_bel = curr_array[row_num+1,col_num-1:col_num+2]
            list_neigh_values.extend(row_abv.tolist())
            list_neigh_values.extend(row_bel.tolist())
            list_idx_scale = [c-1, c+1]
            for scale_idx in list_idx_scale:
                scale_DoG = list_DOG[scale_idx]
                arr_slice = scale_DoG[row_num-1:row_num+2, col_num-1:col_num+2]
                list_slice = arr_slice.tolist()
                for r in list_slice:
                    list_neigh_values.extend(r)
            
            #Now finding the max and min value in the neighbors
            max_neigh = max(list_neigh_values)
            min_neigh = min(list_neigh_values)
            
            if curr_pixel_val > max_neigh or curr_pixel_val < min_neigh:
                list_max_min_cord.append([row_num, col_num])

In [37]:
#Eliminating the low contrast points among all keypoints
value_k = 0.05
#Now moving window and finding the M matrix for the window
dict_window_cord = {}
list_score_pairs = []
count_window = 0

for d in range(len(list_max_min_cord)):
    c = list_max_min_cord[d]
    c_r, c_c = c[0], c[1]
    sum_ix2, sum_iy2, sum_ix_iy = 0, 0, 0
    val_ix = temp_pad_1[c_r][c_c-1] - temp_pad_1[c_r][c_c+1]
    val_iy = temp_pad_1[c_r-1][c_c] - temp_pad_1[c_r+1][c_c]
    sum_ix2 = sum_ix2 + (val_ix * val_ix)
    sum_iy2 = sum_iy2 + (val_iy * val_iy)
    sum_ix_iy = sum_ix_iy + (val_ix * val_iy)
    val_determinant = (sum_ix2 * sum_iy2) - (sum_ix_iy * sum_ix_iy)
    val_trace = sum_ix2 + sum_iy2
    val_score = val_determinant - (value_k * val_trace)
    list_score_pairs.append([d, val_score])
    
list_score_pairs.sort(key = lambda x:x[1], reverse = True)

In [38]:
count_keyp = 250
copy_img_rgb_array = np.array(img_rgb_array)

for i in range(count_keyp):
    idx_pt = list_score_pairs[i][0]
    key_pt = list_max_min_cord[idx_pt]
    orig_x, orig_y = key_pt[0]-1, key_pt[1]-1
    copy_img_rgb_array[orig_x][orig_y][0] = 255 
    copy_img_rgb_array[orig_x][orig_y][1] = 0 
    copy_img_rgb_array[orig_x][orig_y][2] = 0 
new_img = Image.fromarray(copy_img_rgb_array)
new_img.save("Result_SIFT_keypoints.png")
new_img.show()
